In [83]:
import pandas as pd
from PCA_and_ETC import read_and_preprocess_data, generate_PCA_Data
import numpy as np
import pandas as pd
import yfinance as yf

In [108]:
sp = pd.read_csv('../Database/ETC/sp500.csv', index_col=0)

In [98]:
sp = yf.download('^GSPC', start='1990-01-01')

[*********************100%%**********************]  1 of 1 completed


In [110]:
sp_monthly = sp.resample('M').last()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [ ]:
index = sp_monthly['Adj Close'].pct_change().dropna()

In [102]:
index=index.apply(lambda x: np.log(x+1))

In [103]:

col = ['06/1999-12/1999', '01/2000-12/2002(9/11)', '01/2003-12/2006', '01/2007-12/2009(GFC)',
       '01/2010-12/2019', '01/2020-12/2022(Covid-19)', 'Overall']

period = [range(0, 119), range(119, 155), range(155, 203), range(203, 239), range(239, 359), range(359, 391),
          range(0, 391)]

In [123]:
row=index.T.iloc[range(203, 239)]

In [132]:
returns = np.exp(np.mean(row) *5.4)-1

In [133]:
returns

-0.03543421351437548

In [21]:
index.T.to_csv('../Database/ETC/sp500_return.csv')

In [135]:
import os
import numpy as np
import pandas as pd
from PCA_and_ETC import read_and_preprocess_data, generate_PCA_Data

cl_list = ['60_characteristics_us', '70_characteristics_us', '80_characteristics_us', '90_characteristics_us']
# cl_list = [f'{i}_characteristics_us' for i in range(10, 80, 10)]
for a in cl_list:
    input_dir = f'../Database/Clustering_Result/Contrastive_Learning/{a}'
    input_dir2 = f'../Database/Clustering_Result/Contrastive_Learning/Prob/{a}'
    output_dir = f'../Database/Clustering_Result/Contrastive_Learning/CL_Pre/{a}'

    files = sorted(filename for filename in os.listdir(input_dir))

    for file in files:
        data = read_and_preprocess_data(input_dir, file)
        prob = read_and_preprocess_data(input_dir2, file)
        prob.index = prob['firms']
        prob = prob.drop(columns='firms')
        prob_filtered = prob[(prob.max(axis=1) <= prob.max(axis=1).quantile(0.80))]
        data.loc[prob_filtered.index, 'clusters'] = 0
        data.to_csv(os.path.join(output_dir, file))

In [15]:


input_dir = '../Database/Clustering_Result/Contrastive_Learning2/CL_pre'
input_dir2 = '../Database/characteristics_US'
subdirectories = [d for d in os.listdir(input_dir)]

for subdir in subdirectories:
    base_directory = f'{input_dir}/{subdir}'
    files = sorted(filename for filename in os.listdir(base_directory) if filename.endswith('.csv'))

    bic_dict = {}

    for file in files:
        df = read_and_preprocess_data(base_directory, file)
        df2 = read_and_preprocess_data(input_dir2, file)
        df2 = generate_PCA_Data(df2)

        n = df.shape[0]
        d = df2.shape[1]

        clusters = {}
        total_length = 0
        for j in range(1 + max(set(df['clusters']))):
            # if j == 0:
            #     continue
            cluster_data = df2.loc[df[df['clusters'] == j].index, 'mom1':]

            # 각 클러스터에 대하여 정규화된 특성들의 var의 평균을 구함
            clusters[j] = cluster_data.var().mean()

        # total variance는 클러스터 전체의 var의 평균을 사용.
        total_variance = pd.DataFrame(clusters.values()).sum()

        # 최대 가능도
        L = np.exp(-0.5 * total_variance)
        # 모델의 자유도
        k = len(clusters)-1
        v = d + np.log(n/k)
        # BIC 계산
        bic = -2 * np.log(L) + v * np.log(n)
        bic_dict[file] = bic

    print(subdir)
    # 전체기간에 대하여 bic score의 평균을 구함
    a = pd.DataFrame(bic_dict.values()).mean()
    print(a)

10_characteristics_us
0    759.379139
dtype: float64
20_characteristics_us
0    757.98723
dtype: float64
30_characteristics_us
0    757.904674
dtype: float64
40_characteristics_us
0    757.921769
dtype: float64
50_characteristics_us
0    757.500136
dtype: float64
60_characteristics_us
0    760.097656
dtype: float64
70_characteristics_us
0    761.83405
dtype: float64


In [3]:
import numpy as np
import os
import numpy as np
import pandas as pd
from PCA_and_ETC import read_and_preprocess_data, generate_PCA_Data
from sklearn.metrics import pairwise_distances

# 클러스터 내 최대 거리 계산 함수
def intra_cluster_distance(data, labels, cluster):
    cluster_points = data[labels == cluster]
    if len(cluster_points) <= 1:
        return 0
    return np.max(pairwise_distances(cluster_points))

# 클러스터 간 최소 거리 계산 함수
def inter_cluster_distance(cluster_centers):
    if len(cluster_centers) <= 1:
        return 0
    distances = pairwise_distances(cluster_centers)
    np.fill_diagonal(distances, np.inf)  # 자신과의 거리를 무한대로 설정하여 최소 거리를 올바르게 계산
    return np.min(distances)

# Dunn Index 계산 함수
def dunn_index(data, labels):
    unique_clusters = np.unique(labels)
    cluster_centers = np.array([data[labels == cluster].mean(axis=0) for cluster in unique_clusters])
    
    intra_distances = [intra_cluster_distance(data, labels, cluster) for cluster in unique_clusters]
    inter_distance = inter_cluster_distance(cluster_centers)
    
    return inter_distance / np.max(intra_distances)

In [12]:
import os
import numpy as np
import pandas as pd
from PCA_and_ETC import read_and_preprocess_data, generate_PCA_Data

input_dir = '../Database/Clustering_Result/DBSCAN'
input_dir2 = '../Database/characteristics_US'
subdirectories = [d for d in os.listdir(input_dir)]

for subdir in subdirectories:
    base_directory = f'{input_dir}/{subdir}'
    files = sorted(filename for filename in os.listdir(base_directory) if filename.endswith('.csv'))

    dunn_dict = {}

    for file in files:
        df = read_and_preprocess_data(base_directory, file)
        df2 = read_and_preprocess_data(input_dir2, file)
        df2 = generate_PCA_Data(df2)

        # 원본 벡터와 클러스터 레이블을 결합
        data = df2.to_numpy()
        labels = df['clusters'].to_numpy()
        
        # Dunn Index 계산
        dunn_dict[file] = dunn_index(data, labels)

    print(subdir)
    # 전체기간에 대하여 dunn의 평균을 구함
    a = pd.DataFrame(dunn_dict.values()).mean()
    print(a)

0.1
0    0.023543
dtype: float64



KeyboardInterrupt



In [14]:
import os
import numpy as np
import pandas as pd
from PCA_and_ETC import read_and_preprocess_data, generate_PCA_Data
from sklearn.metrics import silhouette_score
input_dir = '../Database/Clustering_Result/Contrastive_Learning2/CL_pre'
input_dir2 = '../Database/characteristics_US'
subdirectories = [d for d in os.listdir(input_dir)]

for subdir in subdirectories:
    base_directory = f'{input_dir}/{subdir}'
    files = sorted(filename for filename in os.listdir(base_directory) if filename.endswith('.csv'))

    dunn_dict = {}

    for file in files:
        df = read_and_preprocess_data(base_directory, file)
        df2 = read_and_preprocess_data(input_dir2, file)
        df2 = generate_PCA_Data(df2)

        # 원본 벡터와 클러스터 레이블을 결합
        data = df2.to_numpy()
        labels = df['clusters'].to_numpy()
        
        # Dunn Index 계산
        dunn_dict[file] = silhouette_score(data, labels)

    print(subdir)
    # 전체기간에 대하여 dunn의 평균을 구함
    a = pd.DataFrame(dunn_dict.values()).mean()
    print(a)

10_characteristics_us
0   -0.205846
dtype: float64



KeyboardInterrupt

